# 🧪 Hallucination Evaluation Notebook
This notebook evaluates your model's outputs against known instructions and expected outputs.
It tracks hallucinations, summarizes accuracy, and visualizes the results.
Includes support for pass@k and CSV export.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from transformers import pipeline
from tqdm import tqdm

## 🔍 Load Known Test Cases

In [ ]:
with open('data/hallucination_cases.jsonl') as f:
    cases = json.load(f)['cases']
print(f'Loaded {len(cases)} evaluation cases.')

## 🤖 Load Your Model

In [ ]:
generator = pipeline("text-generation", model="./checkpoints/my_model")

## ✅ Evaluate and Score (Pass@k Enabled)

In [ ]:
results = []
k = 3  # number of generations to sample per prompt

for case in tqdm(cases):
    prompt = case['instruction']
    expected = case['expected']
    completions = generator(prompt, max_new_tokens=128, num_return_sequences=k, do_sample=True)
    outputs = [c['generated_text'] for c in completions]
    passed = any(expected in out for out in outputs)

    results.append({
        'prompt': prompt,
        'expected': expected,
        'outputs': outputs,
        'hallucinated': not passed,
        'reason': case['reason'] if not passed else ''
    })

## 📊 Visualize and Save Summary

In [ ]:
df = pd.DataFrame(results)
display(df[['prompt', 'hallucinated', 'reason']])

hallucination_counts = df['hallucinated'].value_counts()
labels = ['Correct', 'Hallucinated']
values = [hallucination_counts.get(False, 0), hallucination_counts.get(True, 0)]

plt.figure(figsize=(6, 4))
plt.bar(labels, values, color=["#4CAF50", "#F44336"])
plt.title("Hallucination Evaluation Results")
plt.ylabel("Number of Outputs")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print('Hallucination Rate:', df['hallucinated'].mean())

## 💾 Export Results

In [ ]:
Path('logs').mkdir(exist_ok=True)
df.to_json('logs/hallucination_eval_results.json', indent=2)
df.to_csv('logs/hallucination_eval_results.csv', index=False)
print('✅ Exported results to logs/hallucination_eval_results.*')